In [12]:
from collections import defaultdict, deque
from typing import Set, Dict, Tuple, FrozenSet


class DFAState:
    def __init__(self, state_set: FrozenSet[str], is_final: bool = False):
        self.state_set = state_set  # The set of NFA states this DFA state represents
        self.is_final = is_final    # Whether this DFA state is a final state

    def __repr__(self):
        s = f"({{{', '.join(sorted(self.state_set))}}})"
        if self.is_final:
            s = f"{s[:len(s) - 1]}, {self.is_final})"
        return s


class DFA:
    def __init__(self, sigma: Set[str]):
        self.sigma = sigma
        self.states: Dict[FrozenSet[str], DFAState] = {}  # DFA states indexed by their frozenset of NFA states
        self.start: DFAState = None                      # The DFA start state
        self.transitions: Dict[Tuple[DFAState, str], DFAState] = {}  # DFA transitions

    def add_state(self, state_set: FrozenSet[str], is_final: bool = False):
        if state_set not in self.states:
            self.states[state_set] = DFAState(state_set, is_final)
        return self.states[state_set]

    def add_transition(self, from_state: DFAState, symbol: str, to_state: DFAState):
        self.transitions[(from_state, symbol)] = to_state

    def __repr__(self):
        transition_str = "\n".join(
            [f"  {from_state} --{symbol}--> {to_state}"
             for (from_state, symbol), to_state in self.transitions.items()]
        )
        return (f"DFA(\n"
                f"  Sigma: {self.sigma}\n"
                f"  States: {list(self.states.values())}\n"
                f"  Start State: {self.start}\n"
                f"  Transitions:\n{transition_str}\n"
                f")")


class NFA:
    def __init__(self, 
                 sigma: Set[str], 
                 states: Set[str], 
                 start: str, 
                 finals: Set[str], 
                 transitions: Dict[Tuple[str, str], Set[str]]):
        self.sigma = sigma
        self.states = states
        self.start = start
        self.finals = finals
        self.transitions = transitions  # (state, symbol) -> set of states

    def add_transition(self, from_state: str, symbol: str, to_state: str):
        if (from_state, symbol) not in self.transitions:
            self.transitions[(from_state, symbol)] = set()
        self.transitions[(from_state, symbol)].add(to_state)

    def __repr__(self):
        transition_str = "\n".join(
            [f"  {from_state} --{symbol}--> {', '.join(to_states)}"
             for (from_state, symbol), to_states in sorted(self.transitions.items())]
        )
        return (f"NFA(\n"
                f"  Sigma: {self.sigma}\n"
                f"  States: {self.states}\n"
                f"  Start State: {self.start}\n"
                f"  Final States: {self.finals}\n"
                f"  Transitions:\n{transition_str}\n"
                f")")


def convert_nfa_to_dfa(nfa: NFA) -> DFA:
    dfa = DFA(nfa.sigma)

    # Helper function to compute epsilon-closure
    def epsilon_closure(states: Set[str]) -> Set[str]:
        stack = list(states)
        closure = set(states)
        while stack:
            state = stack.pop()
            for next_state in nfa.transitions.get((state, ''), set()):
                if next_state not in closure:
                    closure.add(next_state)
                    stack.append(next_state)
        return closure

    # Helper function to move on symbol
    def move(states: Set[str], symbol: str) -> Set[str]:
        result = set()
        for state in states:
            result.update(nfa.transitions.get((state, symbol), set()))
        return result

    # Initialize DFA start state
    start_closure = frozenset(epsilon_closure({nfa.start}))
    dfa_start = dfa.add_state(start_closure, any(state in nfa.finals for state in start_closure))
    dfa.start = dfa_start

    # Initialize visited list
    visited = set()
    stack = [dfa_start]

    while stack:
        current_dfa_state = stack.pop()
        if current_dfa_state.state_set in visited:
            continue
        visited.add(current_dfa_state.state_set)

        for symbol in nfa.sigma:
            moved = move(current_dfa_state.state_set, symbol)
            closure = frozenset(epsilon_closure(moved))
            if closure:
                is_final = any(state in nfa.finals for state in closure)
                new_dfa_state = dfa.add_state(closure, is_final)
                dfa.add_transition(current_dfa_state, symbol, new_dfa_state)
                if closure not in visited:
                    stack.append(new_dfa_state)

    return dfa



In [13]:
def example_nfa():
    # Define the NFA components
    sigma = {'a', 'b', 'c'}
    states = {'0', '1', '2'}
    start = '0'
    finals = {'2'}
    transitions = defaultdict(set)

    # Define transitions based on the diagram             # 'a' transition from state 0 to state 1
    transitions[('0', 'a')].add('0')              # 'b' transition from state 0 to state 2
    transitions[('0', 'a')].add('1')              # 'b' transition from state 1 to state 3
    transitions[('1', 'b')].add('1')               # ε-transition from state 1 to state 2
    transitions[('1', 'b')].add('2')               # ε-transition from state 2 to state 1
    transitions[('2', 'c')].add('1')              # 'a' transition from state 2 to state 4
    transitions[('2', 'c')].add('2')              # 'a' transition from state 2 to state 4

    return NFA(sigma, states, start, finals, transitions)


# Test the NFA and DFA
if __name__ == "__main__":
    nfa = example_nfa()
    print("=== NFA ===")
    print(nfa)

    dfa = convert_nfa_to_dfa(nfa)
    print("\n=== DFA ===")
    print(dfa)

=== NFA ===
NFA(
  Sigma: {'a', 'b', 'c'}
  States: {'0', '1', '2'}
  Start State: 0
  Final States: {'2'}
  Transitions:
  0 --a--> 0, 1
  1 --b--> 1, 2
  2 --c--> 1, 2
)

=== DFA ===
DFA(
  Sigma: {'a', 'b', 'c'}
  States: [({0}), ({0, 1}), ({1, 2}, True)]
  Start State: ({0})
  Transitions:
  ({0}) --a--> ({0, 1})
  ({0, 1}) --a--> ({0, 1})
  ({0, 1}) --b--> ({1, 2}, True)
  ({1, 2}, True) --b--> ({1, 2}, True)
  ({1, 2}, True) --c--> ({1, 2}, True)
)
